### This notebook  regards the classification model used for my capstone project

#### The machine learning model I use is RandomForestClassifier of the lib sklearn

In [33]:
import nltk
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
import seaborn as sns
import pandas as pd
from pandas.compat import StringIO
from wordcloud import WordCloud

import string 
import re
from gensim.models import Word2Vec

from nltk.tokenize import RegexpTokenizer

from nltk.tokenize import word_tokenize
from nltk import ne_chunk_sents
from nltk.tokenize import sent_tokenize
from nltk import pos_tag
from wordcloud import WordCloud
pd.option_context('display.max_colwidth', 250)

In [34]:
data = pd.read_csv('job_skills.csv')
data.shape

(1250, 7)

In [35]:
# there are 20 rows containing NaNs in the dataset which are dropped
data2 = data.dropna()
data2.shape

(1235, 7)

#### In order to build a column of labels (otherwise called targets) the Category field of my dataset is encoded in numbers form 0 to 10. Here I'm encoding the first ten most large categories while considering the remaining categories which are less important as a unique category.

In [36]:
my_category= []
categories = {}

for i in data2['Category']:
    if 'Sales & Account Management' in i:
        my_category.append(0)
        categories[0] = "Sales & Account Management"     #thisdict["color"] = "red"
        continue
        
    if 'Marketing & Communications' in i :
        my_category.append(1)
        categories[1] = "Marketing & Communications"
        continue
        
    if 'Finance' in i :
        my_category.append(2)
        categories[2] = "Finance"
        continue
        
    if 'Technical Solutions' in i :
        my_category.append(3)  
        categories[3] = "Technical Solutions"
        continue
        
    if 'Business Strategy' in i :
        my_category.append(4) 
        categories[4] = "Business Strategy"
        continue
        
    if 'People Operations' in i:
        my_category.append(5)
        categories[5] = "People Operations"
        continue
        
    if 'User Experience & Design' in i:
        my_category.append(6)
        categories[6] = "User Experience & Design"
        continue
        
    if 'Program Management' in i:
        my_category.append(7)
        categories[7] = "Program Management"
        continue
        
    if 'Partnerships' in i:
        my_category.append(8)
        categories[8] = "Partnerships"
        continue
        
    if 'Product & Customer Support' in i:
        my_category.append(9)
        categories[9] = "Product & Customer Support"
        
    else:
        my_category.append(10)
        categories[10] = "Other"

In [37]:
# add the column 'My_category' as a label to the dataste
data2.insert(loc=7, column='My_category', value=my_category)

In [38]:
categories

{7: 'Program Management',
 10: 'Other',
 3: 'Technical Solutions',
 8: 'Partnerships',
 9: 'Product & Customer Support',
 4: 'Business Strategy',
 1: 'Marketing & Communications',
 0: 'Sales & Account Management',
 2: 'Finance',
 6: 'User Experience & Design',
 5: 'People Operations'}

In [39]:
# here a small print-out just to check the label column is created correctly
data2.head(2)

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications,My_category
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...,7
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin...",10


### Vectorization : it is a way of uncovering the underlying structure of the dataset. This structure can then be learned and used to identify similar structures in other dataset. 

### As a way of vectorization I'm using a Term Frequency - Inverse Document Frequency Matrix implemented in TfidfVectorizer class of sklearn 

#### The TF-IDF creates a document term matrix, where there's one row per text and the columns represent single unique terms. But the cells do not represent the count, they represent instead a weighting that's meant to identify how important a word is to an individual text. The formula below shows how this weighting is computed. 

<img src="files/tfidf.png">

### In order to build the TF-IDF matrix the following steps will be performed:
#### 1- The text in 'Responsibilities' column in the dataset will be tokenized, punctuation will be removed, il will     be stemed in order to find the root of the words and stop words will be also removed
#### 2- I have implemented a function that does just that-
#### 3- the function will then be passed to the class TfidfVectorizer that will use it to (see 4)
#### 4- create the TF-IDF matrix by using the fit_transform() function.

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [95]:
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [96]:
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_txt(tex):
    tex_no_pun ="".join([char for char in tex if char not in string.punctuation])
    tokens = re.split('\W+', tex_no_pun)
    tex_no_pun = [ps.stem(word) for word in tokens if word not in stopwords]
    return tex_no_pun

sentences = []
for i in data2['Responsibilities']:
    li = clean_txt(i)
    sentences.append(li)

tfidf_vect = TfidfVectorizer(analyzer=clean_txt)

In [97]:
X_tfidf = tfidf_vect.fit_transform(data2['Responsibilities'])
print(X_tfidf.shape)
print(type(X_tfidf))
print(tfidf_vect.get_feature_names())

(1235, 2775)
<class 'scipy.sparse.csr.csr_matrix'>
['', '10q10k', '10x', '11', '1tomani', '20', '247', '24x7', '2D', '2d3d', '30', '3D', '3plasp', '3pr', '40', '50', 'AB', 'AM', 'AR', 'AV', 'Ad', 'As', 'BD', 'BI', 'Be', 'C', 'CA', 'CC', 'CE', 'CM', 'CX', 'DC', 'EU', 'FA', 'FX', 'G', 'Go', 'HR', 'IA', 'IC', 'IN', 'IP', 'IT', 'If', 'In', 'MA', 'ML', 'OS', 'PA', 'PD', 'PM', 'PR', 'QA', 'SP', 'TV', 'UI', 'UK', 'UL', 'US', 'UX', 'VP', 'VR', 'abil', 'abl', 'abreast', 'abroad', 'absenc', 'abstract', 'abus', 'abusespam', 'academia', 'acceler', 'accept', 'acceptancedeclin', 'access', 'accessori', 'accommod', 'accommodationsaccess', 'accomplish', 'accord', 'accordingli', 'account', 'accountterritori', 'accrual', 'accur', 'accuraci', 'accuratetlmfunct', 'achiev', 'achiv', 'acquir', 'acquisit', 'across', 'act', 'action', 'actionsimprov', 'activ', 'actual', 'acumen', 'ad', 'adapt', 'add', 'addit', 'address', 'adequ', 'adher', 'adhoc', 'adjust', 'administ', 'administr', 'admiss', 'admob', 'adopt', '

#### In the next cell, I create a dataframe with the values of the TF-IDF matrix that will be needed for creating my training and test dataset. The feature names of the matrix are asigned as columns to the dataframe.  

In [98]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns = tfidf_vect.get_feature_names()

#### The following code will import the utility method train_test_split of sklearn.model_selection that will help me divide my dataset in training and testing part. Also precision_recall_fscore_support is imported in order to get the performance scores of my model

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_df, data2['My_category'], test_size= 0.2)

### My problem is a multiclass classification and I will use a RandomForestClassifier 
#### The RandomForestClassifier 

In [101]:
from sklearn.ensemble import RandomForestClassifier

In [102]:
rf = RandomForestClassifier(n_estimators=50, max_depth=None, n_jobs = -1)

#### The cross validation will help detect if there is a variance problem with the accuracy. The variance problem refers to the case in which the accuracy obtained on one test set is very different to the accuracy obtained on another test set using the same algorithm. This behavior is a problem as it brings uncertainty into the process. 

In [103]:
from sklearn.model_selection import KFold, cross_val_score

In [104]:
fold_5_accuracies = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5, scoring='accuracy')  

In [105]:
fold_5_accuracies

array([0.74752475, 0.76      , 0.8071066 , 0.72959184, 0.73056995])

In [106]:
type(fold_5_accuracies)

numpy.ndarray

In [107]:
print("The mean and std of accuracy variance = "+str(fold_5_accuracies.mean())+","+str(fold_5_accuracies.std()))

The mean and std of accuracy variance = 0.7549586272762483,0.028419608079388124


#### The standard deviation of accuracy is pretty small and this means that the model has a small variance. This means also that the prediction I will obtain on the test set will not be by chance but it will be varying only a little bit.  

### Fit the model and find out most important features

In [108]:
rf_model = rf.fit(X_train, y_train)

In [109]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse = True)[0:20]

[(0.011155791916469149, 'intern'),
 (0.010685602712047293, 'partner'),
 (0.009454629774904175, 'respons'),
 (0.009187888516980732, 'opportun'),
 (0.008916603822282078, 'market'),
 (0.008814977471989107, 'technic'),
 (0.008345585296317229, 'design'),
 (0.008130482958069306, 'develop'),
 (0.008124453740652264, 'account'),
 (0.007781174826090262, 'busi'),
 (0.0076280843350508476, 'financi'),
 (0.007622032244858429, 'give'),
 (0.007134267731876356, 'custom'),
 (0.007125677748699116, 'strateg'),
 (0.007064061515315377, 'googl'),
 (0.007003574708178064, 'product'),
 (0.006808162703409063, 'user'),
 (0.00618588045333642, 'work'),
 (0.00603626047407941, 'candid'),
 (0.005974680631710202, 'relationship')]

#### Making predictions using measures like precision, recall, fscore 


####  <img src="files/Precisionrecall.svg.png" alt="PrecisionRecall" height="250" width="300">

In [110]:
y_pred = rf_model.predict(X_test)

In [111]:
precision, recall, fscore, support = score(y_test, y_pred)

#### The following code will display the measures of precision, recall and fscore for every category

In [112]:
for i in range(11):
    print() 
    print(str(categories[i])+" ===> "+'Precision :{} / Recall : {} / Fscore : {} '.format(round(precision[i],3), 
                                                                                          round(recall[i],3), 
                                                                                          round(fscore[i],3),
                                                                                          
          )
          )


Sales & Account Management ===> Precision :0.838 / Recall : 0.939 / Fscore : 0.886 

Marketing & Communications ===> Precision :0.81 / Recall : 0.944 / Fscore : 0.872 

Finance ===> Precision :0.882 / Recall : 0.789 / Fscore : 0.833 

Technical Solutions ===> Precision :0.905 / Recall : 0.905 / Fscore : 0.905 

Business Strategy ===> Precision :0.889 / Recall : 0.762 / Fscore : 0.821 

People Operations ===> Precision :0.952 / Recall : 0.909 / Fscore : 0.93 

User Experience & Design ===> Precision :1.0 / Recall : 0.706 / Fscore : 0.828 

Program Management ===> Precision :1.0 / Recall : 0.727 / Fscore : 0.842 

Partnerships ===> Precision :0.8 / Recall : 0.667 / Fscore : 0.727 

Product & Customer Support ===> Precision :1.0 / Recall : 0.3 / Fscore : 0.462 

Other ===> Precision :0.741 / Recall : 0.956 / Fscore : 0.835 


In [113]:
acc = round((y_pred == y_test).sum() / len(y_pred) , 3)
acc

0.846

### Tuning of parameters 

### The measure that I'm trying to improve is F-score. I find this measure is important as it expresses precision and recall in a synthetic way

In [114]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators = n_est, max_depth = depth, n_jobs = -1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision,recall,fscore,support = score(y_test, y_pred)
    print('Est:{} / Depth: {} / Pre :{} / Rec : {} / Fscore : {} / Acc.: {} '.format(
          n_est, depth, precision[0], recall[0], fscore[0], round((y_pred == y_test).sum() / len(y_pred) , 3)
          )) 
    
for n_est in [10,30, 50, 100]:
    for depth in [10,30, 50, 150, None]:
        train_RF(n_est, depth)
    print('---------------------------------------------------')     

Est:10 / Depth: 10 / Pre :0.6842105263157895 / Rec : 0.7878787878787878 / Fscore : 0.732394366197183 / Acc.: 0.619 
Est:10 / Depth: 30 / Pre :0.7837837837837838 / Rec : 0.8787878787878788 / Fscore : 0.8285714285714285 / Acc.: 0.785 
Est:10 / Depth: 50 / Pre :0.7435897435897436 / Rec : 0.8787878787878788 / Fscore : 0.8055555555555556 / Acc.: 0.781 
Est:10 / Depth: 150 / Pre :0.7435897435897436 / Rec : 0.8787878787878788 / Fscore : 0.8055555555555556 / Acc.: 0.773 
Est:10 / Depth: None / Pre :0.7894736842105263 / Rec : 0.9090909090909091 / Fscore : 0.8450704225352113 / Acc.: 0.785 
---------------------------------------------------
Est:30 / Depth: 10 / Pre :0.6511627906976745 / Rec : 0.8484848484848485 / Fscore : 0.736842105263158 / Acc.: 0.664 
Est:30 / Depth: 30 / Pre :0.8157894736842105 / Rec : 0.9393939393939394 / Fscore : 0.8732394366197183 / Acc.: 0.81 
Est:30 / Depth: 50 / Pre :0.7837837837837838 / Rec : 0.8787878787878788 / Fscore : 0.8285714285714285 / Acc.: 0.822 
Est:30 / Dep

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Est:50 / Depth: 10 / Pre :0.7567567567567568 / Rec : 0.8484848484848485 / Fscore : 0.8000000000000002 / Acc.: 0.688 
Est:50 / Depth: 30 / Pre :0.8333333333333334 / Rec : 0.9090909090909091 / Fscore : 0.8695652173913043 / Acc.: 0.814 
Est:50 / Depth: 50 / Pre :0.7692307692307693 / Rec : 0.9090909090909091 / Fscore : 0.8333333333333333 / Acc.: 0.826 
Est:50 / Depth: 150 / Pre :0.8823529411764706 / Rec : 0.9090909090909091 / Fscore : 0.8955223880597014 / Acc.: 0.826 
Est:50 / Depth: None / Pre :0.8157894736842105 / Rec : 0.9393939393939394 / Fscore : 0.8732394366197183 / Acc.: 0.834 
---------------------------------------------------
Est:100 / Depth: 10 / Pre :0.7631578947368421 / Rec : 0.8787878787878788 / Fscore : 0.8169014084507042 / Acc.: 0.7 
Est:100 / Depth: 30 / Pre :0.9117647058823529 / Rec : 0.9393939393939394 / Fscore : 0.9253731343283583 / Acc.: 0.826 
Est:100 / Depth: 50 / Pre :0.7837837837837838 / Rec : 0.8787878787878788 / Fscore : 0.8285714285714285 / Acc.: 0.822 
Est:100 

#### Finding the best parameters for my model through grid searching and tuning can be a long process . There is a better way to do the things and this is  by using GridSearchCV class of sklearn.model_selection

In [115]:
grid_par = {  
    'n_estimators': [50, 100, 300, 500],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [116]:
from sklearn.model_selection import GridSearchCV 

In [117]:
gd_sr = GridSearchCV(estimator=rf_model,  
                     param_grid=grid_par,
                     scoring='f1_weighted',
                     cv=5,
                     n_jobs=-1)

In [118]:
gd_sr.fit(X_train, y_train)  

/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 300, 500], 'criterion': ['gini', 'entropy'], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [119]:
best_param= gd_sr.best_params_  
print(best_param)  

{'bootstrap': False, 'criterion': 'gini', 'n_estimators': 300}


In [120]:
gd_sr.predict(X_test)

array([ 0,  0,  1,  5,  3,  6,  5,  3,  8,  8,  9, 10,  3,  0,  3,  1,  8,
        0,  0, 10,  9,  3,  1,  4, 10, 10, 10,  2,  3,  1,  1,  4,  0,  1,
       10,  1,  2,  1, 10,  0,  0, 10,  3,  5,  0,  2,  1, 10, 10,  6, 10,
        0,  1,  3,  4,  1,  2, 10,  6,  6,  1, 10,  0,  5,  4,  2,  0,  8,
       10,  3,  4,  5, 10, 10,  2,  4,  2,  1, 10,  8,  7,  0,  6,  3,  3,
        3, 10,  5,  1,  2,  5,  0,  1,  3, 10,  6,  0, 10,  1,  0,  4,  0,
        1,  3,  5,  5,  3,  9,  6,  7, 10,  0, 10, 10,  1,  5,  4,  0,  1,
        4, 10,  1,  1,  6,  5,  1,  7, 10,  8,  2,  8,  6, 10,  1, 10, 10,
        0,  5, 10,  0,  3, 10, 10,  2,  7, 10,  1,  3, 10,  1,  2, 10, 10,
        0,  8,  1,  0,  1,  3,  1,  1,  4,  7, 10,  4,  4,  0, 10,  0,  3,
        7,  0, 10,  5,  7, 10,  5,  0,  0,  2,  0,  1,  4,  1,  5,  3,  4,
        1, 10, 10,  0,  1,  0,  2,  1,  4,  4, 10, 10,  1, 10,  4,  2,  3,
        5,  2,  0, 10,  8,  2,  5,  0, 10,  1,  8, 10, 10, 10, 10,  1,  6,
        2, 10,  5,  0, 10

### Now that I have the best hyperparameter I can check and see if the predictions improve. 

In [121]:
y_pred_bis = gd_sr.predict(X_test)

In [122]:
precision, recall, fscore, support = score(y_test, y_pred_bis)

In [126]:
for i in range(11):
    print()
    print(str(categories[i])+" b ===> "+'Precision :{} / Recall : {} / Fscore : {} '.format(
           round(precision[i],3), round(recall[i],3), round(fscore[i],3)
          )) 


Sales & Account Management b ===> Precision :0.789 / Recall : 0.909 / Fscore : 0.845 

Marketing & Communications b ===> Precision :0.854 / Recall : 0.972 / Fscore : 0.909 

Finance b ===> Precision :0.778 / Recall : 0.737 / Fscore : 0.757 

Technical Solutions b ===> Precision :0.905 / Recall : 0.905 / Fscore : 0.905 

Business Strategy b ===> Precision :0.842 / Recall : 0.762 / Fscore : 0.8 

People Operations b ===> Precision :0.952 / Recall : 0.909 / Fscore : 0.93 

User Experience & Design b ===> Precision :1.0 / Recall : 0.706 / Fscore : 0.828 

Program Management b ===> Precision :1.0 / Recall : 0.636 / Fscore : 0.778 

Partnerships b ===> Precision :0.8 / Recall : 0.667 / Fscore : 0.727 

Product & Customer Support b ===> Precision :1.0 / Recall : 0.3 / Fscore : 0.462 

Other b ===> Precision :0.754 / Recall : 0.956 / Fscore : 0.843 


### In order to check if the measure Fscore improves by using GridSearchCV I confront the Fscore value that I get with GridSearchCV with the values I obtained previously 

### Previous results : screenshot 
<img src="files/previousresults.png">

### Analysis of the results from confrontation between the two versions.

#### By using GridSearchCV the Fscore measure improves in 8 out of 11 categories. For the remaining 3 categories the Fscore worsens. 